In [1]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt 
from IPython import display
import glob
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import cv2
from os import listdir
from sklearn.utils import shuffle

from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [9]:
def scale_and_normalize(arr):
    arr = arr.astype('float32')
    mean, stand_dev = arr.mean(), arr.std()
    arr = (arr-mean)/stand_dev
    arr = np.clip(arr, -1, 1)
    arr = (arr+1)/2
    return arr

In [10]:
def load_data(dir_list, image_size):
    """
    Read images, resize and normalize them. 
    Arguments:
        dir_list: list of strings representing file directories.
    Returns:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """

    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in listdir(directory):
            # load the image
            image = cv2.imread(directory + '/' + filename)
            # crop the brain and ignore the unnecessary rest part of the image
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # resize image
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            # normalize values
            image = scale_and_normalize(image)
            # convert image to numpy array and append it to X
            X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
            if directory[-3:] == 'yes':
                y.append([1])
            else:
                y.append([0])
                
    X = np.array(X)
    y = np.array(y)
    
    # Shuffle the data
    X, y = shuffle(X, y)
    
    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')
    
    return X, y

In [11]:
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X, y = load_data(['../data/JPG_Brain_Scans/yes', '../data/JPG_Brain_Scans/no'], (IMG_WIDTH, IMG_HEIGHT))

Number of examples is: 253
X shape is: (253, 240, 240)
y shape is: (253, 1)


In [13]:
X = X.reshape(-1, 240, 240, 1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, shuffle=True, stratify=y)

In [32]:
data_augmentation = keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [108]:
# train_gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=30,
#                                                          horizontal_flip=True,
#                                                          vertical_flip=True)
# # valid_gen = ImageDataGenerator().flow_from_directory(valid_dir)

# train_gen.fit(X_train)

In [15]:
model = keras.models.Sequential([
    keras.Input(shape=(240, 240, 1)),
#     data_augmentation,
    keras.layers.Conv2D(32, 3, strides=(1, 1), activation='relu', data_format='channels_last', name='conv0'),
    keras.layers.MaxPool2D((2, 2), name='max_pool0'),
    keras.layers.BatchNormalization(name='bn0'),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation='sigmoid')])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 238, 238, 32)      320       
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 119, 119, 32)      0         
_________________________________________________________________
bn0 (BatchNormalization)     (None, 119, 119, 32)      128       
_________________________________________________________________
flatten (Flatten)            (None, 453152)            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 453153    
Total params: 453,601
Trainable params: 453,537
Non-trainable params: 64
_________________________________________________________________


In [17]:
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss = 'binary_crossentropy',
             optimizer = opt, metrics = ['binary_accuracy'])

In [18]:
from datetime import datetime
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

neural_net = model.fit(X_train, y_train, batch_size=32, epochs = 10, validation_data = (X_test, y_test), callbacks=[tensorboard_callback])

Epoch 1/10
6/6 [==============================] - 7s 974ms/step - loss: 19.5429 - binary_accuracy: 0.5705 - val_loss: 0.6011 - val_binary_accuracy: 0.8095
Epoch 2/10
6/6 [==============================] - 5s 766ms/step - loss: 10.4421 - binary_accuracy: 0.7843 - val_loss: 1.5950 - val_binary_accuracy: 0.6667
Epoch 3/10
6/6 [==============================] - 5s 764ms/step - loss: 5.8355 - binary_accuracy: 0.8193 - val_loss: 1.9031 - val_binary_accuracy: 0.4643
Epoch 4/10
6/6 [==============================] - 5s 766ms/step - loss: 2.8892 - binary_accuracy: 0.8737 - val_loss: 0.6487 - val_binary_accuracy: 0.6786
Epoch 5/10
6/6 [==============================] - 6s 969ms/step - loss: 2.0446 - binary_accuracy: 0.8668 - val_loss: 0.5493 - val_binary_accuracy: 0.7619
Epoch 6/10
6/6 [==============================] - 5s 760ms/step - loss: 0.6230 - binary_accuracy: 0.9358 - val_loss: 0.3979 - val_binary_accuracy: 0.8214
Epoch 7/10
6/6 [==============================] - 5s 811ms/step - loss: 0.

In [19]:
sum(y_test)/len(y_test)

array([0.60714286])